In [1]:
import os
import json
import itertools

## ScienceQA - Chameleon ChatGPT

In [2]:
result_file = "../../results/scienceqa/chameleon_chatgpt_test_cache.jsonl"
data_file = "../../data/scienceqa/problems.json"

prob_data = json.load(open(data_file))

with open(result_file) as f:
    results = {}
    for line in f:
        try:
            data = json.loads(line)
            results[data["pid"]] = data
        except:
            pass

In [3]:
modules_list = []

for pid, result in results.items():
    modules = result["modules:output"]
    if "program_generator_and_verifier" in modules:
        index = modules.index("program_generator_and_verifier")
        nested = []
        for module in modules:
            if module == "program_generator_and_verifier":
                nested.append(["program_generator", "program_verifier"])
            else:
                nested.append([module])
        modules = [m for sublist in nested for m in sublist]
        #print(modules)
    
    modules = ["START"] + modules + ["END"]
    modules_list.append(modules)

In [4]:
# modules_list

In [5]:
transition_num = {
 "START": {},
 'knowledge_retrieval': {},
 'text_detector': {},
 'query_generator': {},
 'bing_search': {},
 'image_captioner': {},
  'solution_generator': {},
 'answer_generator': {},
}


transitions = {
 "START": {},
 'knowledge_retrieval': {},
 'text_detector': {},
 'query_generator': {},
 'bing_search': {},
 'image_captioner': {},
  'solution_generator': {},
 'answer_generator': {},
}

In [6]:
for modules in modules_list:
    for i, module in enumerate(modules[:-1]):
        next_module = modules[i+1]
        if next_module not in transition_num[module]:
            transition_num[module][next_module] = 1
        else:
            transition_num[module][next_module] += 1
            
        transitions[module][next_module] = round(transition_num[module][next_module] / sum(transition_num[module].values()), 2)

In [7]:
transition_num

{'START': {'knowledge_retrieval': 2464,
  'text_detector': 726,
  'image_captioner': 330,
  'query_generator': 82,
  'solution_generator': 639},
 'knowledge_retrieval': {'solution_generator': 3027,
  'bing_search': 16,
  'image_captioner': 3},
 'text_detector': {'solution_generator': 243,
  'knowledge_retrieval': 409,
  'image_captioner': 74},
 'query_generator': {'bing_search': 82},
 'bing_search': {'solution_generator': 123, 'image_captioner': 1},
 'image_captioner': {'knowledge_retrieval': 173,
  'solution_generator': 209,
  'bing_search': 26},
 'solution_generator': {'answer_generator': 4241},
 'answer_generator': {'END': 4241}}

In [8]:
transitions

{'START': {'knowledge_retrieval': 0.58,
  'text_detector': 0.17,
  'image_captioner': 0.08,
  'query_generator': 0.02,
  'solution_generator': 0.15},
 'knowledge_retrieval': {'solution_generator': 0.99,
  'bing_search': 0.01,
  'image_captioner': 0.0},
 'text_detector': {'solution_generator': 0.34,
  'knowledge_retrieval': 0.56,
  'image_captioner': 0.1},
 'query_generator': {'bing_search': 1.0},
 'bing_search': {'solution_generator': 0.99, 'image_captioner': 0.01},
 'image_captioner': {'knowledge_retrieval': 0.43,
  'solution_generator': 0.51,
  'bing_search': 0.06},
 'solution_generator': {'answer_generator': 1.0},
 'answer_generator': {'END': 1.0}}

In [9]:
for module in transitions:
    print(module.replace("_", " "))
    
    next_module = transitions[module]
    for m in next_module:
        print(f" -> {m}: {next_module[m]}".replace("_", " "))
        
    print("")

START
 -> knowledge retrieval: 0.58
 -> text detector: 0.17
 -> image captioner: 0.08
 -> query generator: 0.02
 -> solution generator: 0.15

knowledge retrieval
 -> solution generator: 0.99
 -> bing search: 0.01
 -> image captioner: 0.0

text detector
 -> solution generator: 0.34
 -> knowledge retrieval: 0.56
 -> image captioner: 0.1

query generator
 -> bing search: 1.0

bing search
 -> solution generator: 0.99
 -> image captioner: 0.01

image captioner
 -> knowledge retrieval: 0.43
 -> solution generator: 0.51
 -> bing search: 0.06

solution generator
 -> answer generator: 1.0

answer generator
 -> END: 1.0

